# Variables

In [1]:
YOUR_NAME = 'sara'

AWS_PROFILE = 'cities'

# Setup

In [2]:
%load_ext autoreload

In [3]:
MAIN_PATH = "s3://wri-cities-sandbox/identifyingLandSubdivisions/data"
INPUT_PATH = f'{MAIN_PATH}/input'
CITY_INFO_PATH = f'{INPUT_PATH}/city_info'
EXTENTS_PATH = f'{CITY_INFO_PATH}/extents'
BUILDINGS_PATH = f'{INPUT_PATH}/buildings'
ROADS_PATH = f'{INPUT_PATH}/roads'
INTERSECTIONS_PATH = f'{INPUT_PATH}/intersections'
GRIDS_PATH = f'{INPUT_PATH}/city_info/grids'
OUTPUT_PATH = f'{MAIN_PATH}/output'
OUTPUT_PATH_CSV = f'{OUTPUT_PATH}/csv'
OUTPUT_PATH_RASTER = f'{OUTPUT_PATH}/raster'
OUTPUT_PATH_PNG = f'{OUTPUT_PATH}/png'
OUTPUT_PATH_RAW = f'{OUTPUT_PATH}/raw_results'

In [4]:
# Check s3 connection using AWS_PROFILE=CitiesUserPermissionSet profile 
import boto3

session = boto3.Session(profile_name=AWS_PROFILE)
s3 = session.client('s3')

# export CitiesUserPermissionSet profile to use in the next cells
import os
os.environ['AWS_PROFILE'] = AWS_PROFILE


s3.list_buckets()

{'ResponseMetadata': {'RequestId': '9DGQRYRNRVRAWJAW',
  'HostId': 'lH6gi4zpv8uf+Rss52sNvlPuaKMqKD7qxVj6cvKTkxz2cjEgtJpVgSxp8t81m9BIHmk6V7Nb0Kc=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'lH6gi4zpv8uf+Rss52sNvlPuaKMqKD7qxVj6cvKTkxz2cjEgtJpVgSxp8t81m9BIHmk6V7Nb0Kc=',
   'x-amz-request-id': '9DGQRYRNRVRAWJAW',
   'date': 'Thu, 28 Aug 2025 22:46:37 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': 'aft-sandbox-540362055257',
   'CreationDate': datetime.datetime(2022, 9, 13, 15, 12, 20, tzinfo=tzutc())},
  {'Name': 'amplify-citiesindicatorsapi-dev-10508-deployment',
   'CreationDate': datetime.datetime(2023, 8, 30, 5, 5, 13, tzinfo=tzutc())},
  {'Name': 'cities-heat',
   'CreationDate': datetime.datetime(2023, 6, 1, 13, 22, 1, tzinfo=tzutc())},
  {'Name': 'cities-test-sandbox',
   'CreationDate': datetime.datetime(2025, 4, 18, 19, 10, 49, tzinfo=tzutc())},
  {'Name': 'do-not

In [ ]:
import coiled

cluster = coiled.Cluster(
    workspace="wri-cities-data",
    name=f'ils-{YOUR_NAME}',
    region="us-west-2",
    arm=True,
    worker_vm_types="r8g.xlarge",
    spot_policy="spot",
    n_workers=4,
)
client = cluster.get_client()

print(f"Started a new Dask client on Coiled. Dashboard is available at {client.dashboard_link}")


[2025-08-28 17:46:38,432][INFO    ][coiled] Using existing cluster: 'ils-sara (id: 1120426)'
[2025-08-28 17:46:38,433][INFO    ][coiled] Attaching to existing cluster (name: ils-sara, https://cloud.coiled.io/clusters/1120426 )


Started a new Dask client on Coiled. Dashboard is available at https://cluster-aznvp.dask.host/1L-miluBiDBfSv4q/status


2025-08-28 18:08:58,172 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
2025-08-28 18:11:10,991 - distributed.deploy.cluster - WARNING - Failed to sync cluster info multiple times - perhaps there's a connection issue? Error:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/subdivisions2/lib/python3.12/site-packages/distributed/utils.py", line 1923, in wait_for
    return await fut
           ^^^^^^^^^
  File "/opt/anaconda3/envs/subdivisions2/lib/python3.12/site-packages/distributed/comm/tcp.py", line 547, in connect
    stream = await self.client.connect(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/subdivisions2/lib/python3.12/site-packages/tornado/tcpclient.py", line 279, in connect
    af, addr, stream = await connector.start(connect_timeout=timeout)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
asyncio.exceptions.CancelledError

The above exception was the direct cause 

In [ ]:
import dask_geopandas as dgpd
import geopandas as gpd
import dask.dataframe as dd

# Define S3 file path
s3_path = "s3://your-bucket-name/combined_cities.parquet"

# S3 path of the zipped Parquet file
s3_zip_path = "s3://your-bucket-name/combined_cities.zip"

urban_extent = f'{GRIDS_PATH}/{city_name}/{city_name}_urban_extent.geoparquet'
extent = dgpd.read_parquet(urban_extent)
parquet_filename_inside_zip = "combined_cities.parquet"  # Name inside the ZIP

# Open the zip file from S3
with fsspec.open(s3_zip_path, mode="rb") as s3_file:
    with zipfile.ZipFile(io.BytesIO(s3_file.read()), "r") as z:
        with z.open(parquet_filename_inside_zip) as parquet_file:
            # Read the Parquet file as a Dask DataFrame
            dask_df = dd.read_parquet(parquet_file, engine="pyarrow")

# Read Parquet file as a Dask DataFrame
dask_df = dd.read_parquet(s3_path, engine="pyarrow", storage_options={"anon": False})

# Convert to Dask GeoDataFrame
dask_gdf = dgpd.from_dask_dataframe(dask_df)


PermissionError: An error occurred while calling the read_parquet method registered to the pandas backend.
Original Message: All access to this object has been disabled

In [7]:
client.close()